In [1]:
from bs4 import BeautifulSoup
from fake_useragent import UserAgent
import re, requests, json, time

def getHtml(method, url, params= None, data= None, encoding= 'utf-8'):
    try:
    # headers = {'user-agent':"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.130 Safari/537.36"}
    # req = requests.request(req_type, url, data)
        headers = {'user-agent':str(UserAgent().random)}

        if method == 'get':
            req = requests.get(url, headers= headers, params= params)
        elif method == 'post':
            req = requests.post(url, headers= headers, data= data)
            
        # req=requests.get(url, headers= headers, params= params)
        if req.status_code == 200:
            print('requests -> 200')
            req.encoding = encoding
            return BeautifulSoup(req.text, 'html.parser')
        print('requests -> ', req.status_code)
        return None
    except:
        return None
    
def textClean(splitChar='', text='', loop=1, reRight=True, notReverse = True): #
    myIndex = 1 if notReverse else 0
    myLoop = 0
    myRange = text if notReverse else reversed(text)
    for i in myRange:
        if i == splitChar: 
            myLoop+=1
            if myLoop==loop: 
                if reRight:
                    return text[myIndex:] if notReverse else text[-myIndex:] # 由index至右
                return text[:myIndex] if notReverse else text[:-myIndex] # 由左至index
        myIndex+=1

In [90]:
soup = getHtml(method='get', url = 'https://search.books.com.tw/search/query/key/python/cat/all')

requests -> 200


In [64]:
# start = time.time()

# index = [i['value'] for i in itemSoup.findAll('input', attrs={'type':'checkbox'})]
# img = [i['data-original'].split('/img/')[-1] for i in itemSoup.findAll('img', class_='itemcov')]
# title = [i['title'] for i in itemSoup.findAll('a', {'rel':'mid_name'})]
# pub = [i['title'] for i in itemSoup.findAll('a', {'rel':'mid_publish'})]
# date = [i.split(':')[-1].strip() for i in itemSoup.findAll(text = re.compile('出版日期'))]
# # author =[i['title'] for i in itemSoup.findAll('a', {'rel':'go_author'})]
# author = []
# for i in itemSoup.findAll('li', class_='item'):
#     author.append(', '.join([j['title'] for j in i.findAll('a', {'rel':'go_author'})]))
# price =[i.findAll('strong')[-1].b.text for i in itemSoup.findAll('span', class_='price')]
# print(time.time()-start)

0.041139841079711914


In [99]:
# Currently using
start = time.time()
data = []
# try:
for i in soup.find('ul', class_='searchbook').findAll('li', class_="item"):
    data.append(
            {
                'index': i.find('input', {'type': 'checkbox'})['value'],
                'img': i.find('img')['data-original'].split('&w=')[0],
                'title': i.find('h3').a['title'],
                'price': i.find('span', class_='price').findAll('strong')[-1].b.text,
                'date': i.find(text=re.compile('出版日期')).split(':')[-1].strip(),
                'author': ', '.join([j['title'] for j in i.findAll('a', {'rel': 'go_author'})]),
                'pub': i.find('a', {'rel': 'mid_publish'}).text,
            }
        )
# except:
#         print('error')
print(time.time()-start)


0.11973929405212402


In [ ]:
# ----------- tmp django Books ISBN search ------------
def searchISBN(request, key, keyISBN, keyLenght):
    from urllib.parse import quote, unquote
    # key = request.GET.get('key')
    # isISBN = checkISBN(key)
    if keyISBN:
        FindLive = [True, True]
        try:
            # 資料庫有無此書
            from polls.models import Books
            from django.utils import timezone
            # ISBN_Length = isISBN[1]
            print('進入搜查models-> Book')
            if keyLenght == 13:
                print('長度13')
                book = Books.objects.get(ISBN_13=key)
            elif keyLenght == 10:
                print('長度10')
                book = Books.objects.get(ISBN_10=key)
            else:
                print('懷疑有問題(ISBN)!!')

            print('有此資料存在')
            bookInfo = {
                'src': book.Img,
                'title': book.Title,
                'isbn': book.ISBN_10+'\t'+book.ISBN_13,
                'author': book.Author,
                'publisher': book.Publisher,
                'date': book.Date,
            }
            bookInfo['content'] = eval(book.Content)
            print('抓到資料')
            # 是否要更新此書價錢 ( > one day)
            # if (timezone.now() - book.Update).days >= 1: #一天86400秒
            #     print('超過時間 ， 進行更新')
            # book.Price = PRICE(getHtml('https://www.tenlong.com.tw/products/{}?list_name=srh'.format(book.ISBN_13).head))
            # book.Update = timezone
            # book.save()
            # else:
            #     print('沒超過時間')
            # print('現在時間 -> ', timezone.now())
            # print('過去時間 -> ', book.Update)
            # print('時間 = ',(timezone.now() - book.Update).seconds)
            print('正要回傳')
            return render(request, 'Search/search.html', {'userKey': key, 'bookInfo': bookInfo, 'price': eval(book.Price)})
            # return render(request, 'Search/search.html', {'bookInfo': bookInfo})
        except:
            print('\n資料庫 沒有此資料')
            # if Books.objects.get(ISBN_13= key):
            #     print('資料庫有資料!!!!!!! 回傳空直')
            #     return render(request, 'Search/search.html', {'alert':'資料庫錯誤'})
            try:
                print('進入搜索')
                soup = getHtml(
                    'https://www.tenlong.com.tw/products/{}?list_name=srh'.format(quote(key))).head
                try:
                    price = [0, 0]  # [天龍，博客來]$
                    content = {}
                    # ----------天瓏-------------------------
                    tsoup = soup.find('meta', {"name": 'description'})
                    if tsoup:
                        for i in tsoup['content'].split('，'):
                            temp = i.split('：')
                            content[temp[0]] = temp[1]

                        ISBN = [
                            content['ISBN'],
                            soup.find('meta', {"property": 'product:ean'})[
                                'content']
                        ]
                        if len(ISBN[0]) > 10:  # 當10碼為13碼時，更換資料
                            print('ISBN 位置錯誤')
                            temp = ISBN[0]
                            ISBN[0] = ISBN[1]
                            ISBN[1] = temp
                            print('更改後的ISBN -> '+ISBN[0]+" , "+ISBN[1])

                        bookInfo = {  # 天龍資料
                            'src': soup.find('meta', {"property": 'og:image'})['content'].split('https://cf-')[1],
                            'title': content['書名'],
                            'isbn': ISBN[0]+'\t'+ISBN[1],
                            'author': content['作者'],
                            'publisher': content['出版社'],
                            'date': content['出版日期'],
                        }
                        price[0] = soup.find('meta', {"property": 'product:sale_price:amount'})[
                            'content']  # 天龍價錢
                        print('天龍蒐集完成!')
                    else:

                        price[0] = 0
                        FindLive[0] = False

                        print('天龍沒有此書')

                    # ---------博客來---------------------
                    print('正在搜尋博客來')
                    soup = getHtml(
                        'https://search.books.com.tw/search/query/key/{}/cat/all'.format(key))
                    if soup:
                        html = soup.body
                        price[1] = html.find(
                            'span', class_='price').findAll('b')[-1].text

                        html = html.find(
                            'div', class_='cntlisearch08').form.ul.li
                        url = 'http:' + html.find('a')['href']
                        soup = getHtml(url)  # 前往商品頁面
                        if soup:
                            if FindLive[0]:  # 如果天龍有書 那就只搜尋博客來目錄
                                pass
                            else:
                                print('搜尋書的資訊')
                                pass

                            # grid_19 alpha
                            html = soup.find('div', class_="grid_19")
                            titles = []
                            html = html.findAll(
                                'div', class_='mod_b type02_m057 clearfix')
                            contents = [[] for i in range(len(html))]

                            for index, content in enumerate(html):
                                titles.append(content.find('h3').string)
                                for j in content.find('div', class_='content').contents:
                                    #         print(len(j.string))
                                    if j.name != 'br':
                                        if (j.string != None) and (len(j.string) > 1):
                                            contents[index].append(
                                                str(j).strip())
                                    else:
                                        contents[index].append(str(j).strip())
                            bookInfo['content'] = [titles, contents]
                            print('博客來完成')
                    else:
                        print('博客來沒有此書')  # 沒執行到!!!!!!!!!!!!!!!!!!!!!
                        price[1] = 'None'
                        FindLive[1] = False
                        if FindLive[0]:
                            # bookInfo['content']=
                            pass

                        # find('article', class_='content').findAll('section')[2].data
                        titles = []
                        contents = []

                    if FindLive[0] or FindLive[1]:
                        try:  # ---------資料儲存--------------------
                            print('正要儲存資料')
                            Books(
                                Title=bookInfo['title'],
                                Img=bookInfo['src'],
                                ISBN_10=ISBN[0],
                                ISBN_13=ISBN[1],
                                Date=bookInfo['date'],
                                Author=bookInfo['author'],
                                Publisher=bookInfo['publisher'],
                                Content=bookInfo['content'],
                                Price=price
                            ).save()
                            print('資料儲存成功')

                        except:
                            print('無法儲存modles裡')
                        # 完成後所有資料
                        return render(request, 'Search/search.html', {'userKey': key, 'bookInfo': bookInfo, 'price': price})
                    else:
                        return render(request, 'Search/search.html', {'userKey': key})
                except:
                    print('錯誤...')
            except:
                print('不明原因錯誤')
                return render(request, 'Search/search.html', {'alert': '不明原因錯誤'})
    else:
        return render(request, 'Search/search.html', {'alert': 'ISBN 錯誤 '})


In [5]:

resultURL = {
    'books': {
        'baseURL': 'https://search.books.com.tw/search/query/cat/{method}/key/{key}/sort/{sort}/ms2/ms2_1/page/{page}/v/0/',
        'method': {
            'all': 'all',
            'b': '1',
            'e': '6'
        },
        'sort': {
            'default': '1',
            'dateN2O': '5',
            'dateO2N': '6',
            'priceH2L': '7',
            'priceL2H': '8'
        },
    },
    'sanmin': {
        'baseURL': 'https://www.sanmin.com.tw/search/index/?ct={method}&K={key}&ls={sort}&vs=list%20sbs&pi={page}',
        'method': {
            'all': 'K',
            'au': 'AU',
            'pu': 'PU',
            'isbn':'ISBN',
            },
        'sort': {
            'default': 'SD',
            'dateN2O': 'ED',
            'dateO2N': 'EA',
            'priceH2L': 'PD',
            'priceL2H': 'PA'
        }
    },
}

In [13]:
reqPage = 2
reqStore = 'books'
reqSort = 'dateN2O'
reqKey = 'java'
reqMethod = 'e'

tmpURL = resultURL[reqStore]
url = tmpURL['baseURL'].format(
    method= tmpURL['method'][reqMethod],
    key = reqKey,
    sort = tmpURL['sort'][reqSort],
    page = reqPage
)
print(url)

https://search.books.com.tw/search/query/cat/6/key/java/sort/5/ms2/ms2_1/page/2/v/0/
